# Practice 1: Data Science Salaries
In this practice assignment I looked at the Salary dataset provided by randomarnab on [kaggle](https://www.kaggle.com/datasets/arnabchaki/data-science-salaries-2023) and predicted the salary . 

The following

## Made By Jarno Weemen